# Betatron Envelope Equation
- The focusing function $K(s)$ is real and hence the amplitude and phase functions satisfy
$$w'' + K w - \frac{1}{w^3} = 0,$$
$$\psi' = \frac{1}{w^2},$$
where the normalization is choosen such that $w^2$ is exactly the Courant-Snyder $\beta$-function and the Courant-Snyder function $\alpha=-\beta'/2=-w w'$.
- We want to use the numerical formalism in order to calculate the $\beta$-function in the FODO cell instead of a single point value obtained by the matrix formalism;
- We then use the numerical formalism in order to evaluate the average $\beta$-function in the FODO cell and compare this value to arithmetic average between max and min values of the $\beta$-function.

- We setup Python modules below;
- We define the right-hand side of the betatron envelope equation;
- We then define matricies for FODO elements;

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from scipy.integrate import solve_ivp, simps
import numpy as np
import matplotlib.pyplot as plt
plt.style.use(r'PaperDoubleFig.mplstyle')

def rhs(t, y, K):
    """
    The right-hand side of the betatron envelope and phase equations;
    t - the current time;
    y - array of [sigma, sigma', psi]
    K - focusing strength
    """
    sigma = y[0]
    sigmap = y[1]
    psi = y[2]
    dsigma_dt = sigmap
    dsigmap_dt = 1/sigma**3 - K*sigma
    dpsi_dt = 1/sigma**2
    return np.concatenate(([dsigma_dt],
                           [dsigmap_dt],
                           [dpsi_dt]))
"""
sol = solve_ivp(rhs, [0, 4*np.pi],
                np.concatenate(([A0+0j], theta0, eta0)),
                max_step=0.1)
"""
def focus(k, l):
    """
    This function could be used to generate "natural focusing" and Quand focusing(defocusing)
    """
    if k>0:
    
        sr_k = np.sqrt(k)
        c = np.cos(sr_k*l)
        s = np.sin(sr_k*l)
        return np.array([[c, s/sr_k],
                         [-sr_k*s, c]])
    elif k<0:
        sr_k = np.sqrt(-k)
        c = np.cosh(sr_k*l)
        s = np.sinh(sr_k*l)
        return np.array([[c, s/sr_k],
                         [sr_k*s, c]])

def drift(L):
    return np.array([[1, L],
                     [0, 1]])


## Assumed parameters

In [ ]:
# UNDULATOR parameters
XLAMD = 0.0186  # undulator wavelength
ku = 2*np.pi/XLAMD
AW0 = 0.86  # rms Undulator parameter
K = np.sqrt(2)*AW0
# FODO parameters
F1ST = 5
QUADF = 30
FL = 10
QUADD = 30
FD = 10

DRL = 100
# Beam parameters
GAMMA0 = 12e9/0.511e6  # beam energy in mc2
nemit = 0.2e-6  # normalized emittance in m rad, which is equal in x and y

## Derived parameters

In [ ]:
# Focusing quads
lf1st = F1ST*XLAMD
lf = FL*XLAMD
kf = 585*QUADF/GAMMA0
# Defocusing quads
ld = FD*XLAMD
kd = -585*QUADD/GAMMA0
# Undulator focusing
L = DRL*XLAMD
Ku = K**2*ku**2/(2*GAMMA0**2)

## The horizontal plane
- We use matrix formalism in order to find FODO matched solution;
- We use the FODO matched solution as an intitial condition in the numerical calculations;
- We then print numerical values and compare numerical average vs arithmetic average of min/max of the $\beta$-function;

In [ ]:
FODOx = focus(kf, lf1st) @ \
        drift(L) @ focus(kd, ld) @ drift(L) @ \
        focus(kf, lf1st)
phi_c = (np.arccos(np.trace(FODOx)/2))
beta_x = FODOx[0,1]/np.sin(phi_c)

sol = solve_ivp(rhs, [0, lf1st], [np.sqrt(beta_x), 0, 0], args=[kf], max_step=0.001)

z = list(sol.t)
betax = list(sol.y[0]**2)

sol = solve_ivp(rhs, [0, L], [sol.y[i,-1] for i in range(3)], args=[0], max_step=0.01)
z += list(sol.t + z[-1])[1:]
betax += list(sol.y[0]**2)[1:]

sol = solve_ivp(rhs, [0, ld], [sol.y[i,-1] for i in range(3)], args=[kd], max_step=0.001)
z += list(sol.t + z[-1])[1:]
betax += list(sol.y[0]**2)[1:]

sol = solve_ivp(rhs, [0, L], [sol.y[i,-1] for i in range(3)], args=[0], max_step=0.01)
z += list(sol.t + z[-1])[1:]
betax += list(sol.y[0]**2)[1:]

sol = solve_ivp(rhs, [0, lf1st], [sol.y[i,-1] for i in range(3)], args=[kf], max_step=0.001)
plt.plot(sol.t, np.sqrt(sol.y[0]**2*nemit/GAMMA0), '.-')
plt.title('The final quad integration')
plt.xlabel('Integration distance, m')
plt.ylabel(r'$\sigma_x,\ \mu m$')
plt.show()
z += list(sol.t + z[-1])[1:]
betax += list(sol.y[0]**2)[1:]
plt.plot(z, betax)
plt.xlabel('Integration distance, m')
plt.ylabel(r'$\beta_x,\ m$')
plt.xlim([0, z[-1]])
plt.show()

print(f"The phase advance is {sol.y[2,-1]} rad.")
bx_av = simps(betax, z)/z[-1]
print(f"Average x beta function is {bx_av} m.")

print(f"The maximum x beta function is {np.max(betax)} m; the minimum x beta function is {np.min(betax)} m.")
print(f"Arithmetic average beta function is {0.5*(np.max(betax)+np.min(betax))} m.")

In [ ]:
print("Thin lens prediction for average beta:")
phic = 0.27653006629258636
(5+np.cos(phic))*(lf+ld+2*L)/(6*np.sin(phic))

### Matrix formalism repeated
- We show that numerical solution matches our matrix solution in two points;

In [ ]:
M1 = focus(kd, ld/2) @ drift(L) @ focus(kf, lf1st)
M2 = focus(kf, lf1st) @ drift(L) @ focus(kd, ld/2)
FODOx = M2@M1
phi_c = (np.arccos(np.trace(FODOx)/2))
print(f'Phase advance is {phi_c} rad.')
beta_x_max = FODOx[0,1]/np.sin(phi_c)
print(f'The maximum x beta function {beta_x_max} m.')
# reverse order
FODOx = M1@M2
phi_c = (np.arccos(np.trace(FODOx)/2))
print(f'Phase advance is {phi_c} rad.')
beta_x_min = FODOx[0,1]/np.sin(phi_c)
print(f'The minimum x beta function {beta_x_min} m.')

print("Comparing Matrix vs Numerical formalisms:")
print(f"Phase advance prediction error is {sol.y[2,-1]-phi_c} rad.")
print(f"The maximum x beta function prediction error is {np.max(betax)-beta_x_max} m.")
print(f"The minimum x beta function prediction error is {np.min(betax)-beta_x_min} m.")

## The vertical plane
- Natural focusing is included in this plane;
- Undulator averaging is assumed here;
- Exact integration could have been implemented here in order to compare with the undulator average solution.

In [ ]:
FODOy = focus(-kf+Ku, lf1st) @ \
        focus(Ku, L) @ focus(-kd+Ku, ld) @ focus(Ku, L) @ \
        focus(-kf+Ku, lf1st)
phi_c = (np.arccos(np.trace(FODOy)/2))
beta_y = FODOy[0,1]/np.sin(phi_c)

sol = solve_ivp(rhs, [0, lf1st], [np.sqrt(beta_y), 0, 0], args=[-kf+Ku], max_step=0.001)

z = list(sol.t)
betay = list(sol.y[0]**2)

sol = solve_ivp(rhs, [0, L], [sol.y[i,-1] for i in range(3)], args=[Ku], max_step=0.01)
z += list(sol.t + z[-1])[1:]
betay += list(sol.y[0]**2)[1:]

sol = solve_ivp(rhs, [0, ld], [sol.y[i,-1] for i in range(3)], args=[-kd+Ku], max_step=0.001)
z += list(sol.t + z[-1])[1:]
betay += list(sol.y[0]**2)[1:]

sol = solve_ivp(rhs, [0, L], [sol.y[i,-1] for i in range(3)], args=[Ku], max_step=0.01)
z += list(sol.t + z[-1])[1:]
betay += list(sol.y[0]**2)[1:]

sol = solve_ivp(rhs, [0, lf1st], [sol.y[i,-1] for i in range(3)], args=[-kf+Ku], max_step=0.001)
plt.plot(sol.t, np.sqrt(sol.y[0]**2*nemit/GAMMA0), '.-')
plt.title('The final quad integration')
plt.xlabel('Integration distance, m')
plt.ylabel(r'$\sigma_y,\ \mu m$')
plt.show()
z += list(sol.t + z[-1])[1:]
betay += list(sol.y[0]**2)[1:]
plt.plot(z, betay)
plt.xlabel('Integration distance, m')
plt.ylabel(r'$\beta_y,\ m$')
plt.xlim([0, z[-1]])
plt.show()

print(f"The phase advance is {sol.y[2,-1]} rad.")
by_av = simps(betay, z)/z[-1]
print(f"Average y beta function is {by_av} m.")

print(f"The maximum y beta function is {np.max(betay)} m; the minimum y beta function is {np.min(betay)} m.")
print(f"Arithmetic average beta function is {0.5*(np.max(betay)+np.min(betay))} m.")

### Matrix formalism repeated

In [ ]:
M1 = focus(-kd+Ku, ld/2) @ focus(Ku, L) @ focus(-kf+Ku, lf1st)
M2 = focus(-kf+Ku, lf1st) @ focus(Ku, L) @ focus(-kd+Ku, ld/2)
FODOy = M2@M1
phi_c = (np.arccos(np.trace(FODOy)/2))
print(f'Phase advance is {phi_c} rad.')
beta_y_max = FODOy[0,1]/np.sin(phi_c)
print(f'The maximum y beta function {beta_y_max} m.')
# reverse order
FODOy = M1@M2
phi_c = (np.arccos(np.trace(FODOy)/2))
print(f'Phase advance is {phi_c} rad.')
beta_y_min = FODOy[0,1]/np.sin(phi_c)
print(f'The minimum y beta function {beta_y_min} m.')

print("Comparing Matrix vs Numerical formalisms:")
print(f"Phase advance prediction error is {sol.y[2,-1]-phi_c} rad.")
print(f"The maximum y beta function prediction error is {np.max(betay)-beta_y_max} m.")
print(f"The minimum y beta function prediction error is {np.min(betay)-beta_y_min} m.")

In [ ]:
plt.plot(z, betax, label=r'$\beta_x$')
plt.plot(z, betay, label=r'$\beta_y$')
plt.xlim([z[0], z[-1]])
plt.xlabel('FODO coordinate, m')
plt.ylabel(r'$\beta$-function, m')
plt.legend()
plt.show()

## Mean beta function matching

In [ ]:
def cost(kf, kd):
    FODOx = focus(kf, lf1st) @ \
            drift(L) @ focus(kd, ld) @ drift(L) @ \
            focus(kf, lf1st)
    phi_c = (np.arccos(np.trace(FODOx)/2))
    beta_x = FODOx[0,1]/np.sin(phi_c)

    sol = solve_ivp(rhs, [0, lf1st], [np.sqrt(beta_x), 0, 0], args=[kf], max_step=0.001)

    z = list(sol.t)
    betax = list(sol.y[0]**2)

    sol = solve_ivp(rhs, [0, L], [sol.y[i,-1] for i in range(3)], args=[0], max_step=0.01)
    z += list(sol.t + z[-1])[1:]
    betax += list(sol.y[0]**2)[1:]

    sol = solve_ivp(rhs, [0, ld], [sol.y[i,-1] for i in range(3)], args=[kd], max_step=0.001)
    z += list(sol.t + z[-1])[1:]
    betax += list(sol.y[0]**2)[1:]

    sol = solve_ivp(rhs, [0, L], [sol.y[i,-1] for i in range(3)], args=[0], max_step=0.01)
    z += list(sol.t + z[-1])[1:]
    betax += list(sol.y[0]**2)[1:]

    sol = solve_ivp(rhs, [0, lf1st], [sol.y[i,-1] for i in range(3)], args=[kf], max_step=0.001)
    z += list(sol.t + z[-1])[1:]
    betax += list(sol.y[0]**2)[1:]

    bx_av = simps(betax, z)/z[-1]
    FODOy = focus(-kf+Ku, lf1st) @ \
            focus(Ku, L) @ focus(-kd+Ku, ld) @ focus(Ku, L) @ \
            focus(-kf+Ku, lf1st)
    phi_c = (np.arccos(np.trace(FODOy)/2))
    beta_y = FODOy[0,1]/np.sin(phi_c)

    sol = solve_ivp(rhs, [0, lf1st], [np.sqrt(beta_y), 0, 0], args=[-kf+Ku], max_step=0.001)

    z = list(sol.t)
    betay = list(sol.y[0]**2)

    sol = solve_ivp(rhs, [0, L], [sol.y[i,-1] for i in range(3)], args=[Ku], max_step=0.01)
    z += list(sol.t + z[-1])[1:]
    betay += list(sol.y[0]**2)[1:]

    sol = solve_ivp(rhs, [0, ld], [sol.y[i,-1] for i in range(3)], args=[-kd+Ku], max_step=0.001)
    z += list(sol.t + z[-1])[1:]
    betay += list(sol.y[0]**2)[1:]

    sol = solve_ivp(rhs, [0, L], [sol.y[i,-1] for i in range(3)], args=[Ku], max_step=0.01)
    z += list(sol.t + z[-1])[1:]
    betay += list(sol.y[0]**2)[1:]

    sol = solve_ivp(rhs, [0, lf1st], [sol.y[i,-1] for i in range(3)], args=[-kf+Ku], max_step=0.001)
    z += list(sol.t + z[-1])[1:]
    betay += list(sol.y[0]**2)[1:]

    by_av = simps(betay, z)/z[-1]
    return (bx_av-by_av)**2
npcost = np.vectorize(cost)

In [ ]:
xkf = np.linspace(0.99*kf, 1.01*kf, 10)
ykd = np.linspace(0.99*kd, 1.01*kd, 10)
X, Y = np.meshgrid(xkf, ykd)
res = npcost(X, Y)
plt.contourf(X, Y, res)
plt.colorbar()
plt.show()

In [ ]:
from scipy.optimize import minimize_scalar
kf = minimize_scalar(cost, bracket=(0.97*kf, kf, 1.03*kf), args=kd).x
print(f"QUADF={GAMMA0*kf/585}")
print(f"QUADD={GAMMA0*kd/585}")

In [ ]:
M1 = focus(kd, ld/2) @ drift(L) @ focus(kf, lf1st)
M2 = focus(kf, lf1st) @ drift(L) @ focus(kd, ld/2)
FODOx = M2@M1
phi_c = (np.arccos(np.trace(FODOx)/2))
print(phi_c)
beta_x = FODOx[0,1]/np.sin(phi_c)
print(np.sqrt(beta_x*nemit/GAMMA0))
# reverse order
FODOx = M1@M2
phi_c = (np.arccos(np.trace(FODOx)/2))
print(phi_c)
beta_x = FODOx[0,1]/np.sin(phi_c)
print(np.sqrt(beta_x*nemit/GAMMA0))
print('phase advance is the same!')

In [ ]:
M1 = focus(-kd+Ku, ld/2) @ focus(Ku, L) @ focus(-kf+Ku, lf1st)
M2 = focus(-kf+Ku, lf1st) @ focus(Ku, L) @ focus(-kd+Ku, ld/2)
FODOy = M2 @ M1
phi_c = (np.arccos(np.trace(FODOy)/2))
print(phi_c)
beta_y = FODOy[0,1]/np.sin(phi_c)
print(np.sqrt(beta_y*nemit/GAMMA0))
# reverse order
FODOy = M1 @ M2
phi_c = (np.arccos(np.trace(FODOy)/2))
print(phi_c)
beta_y = FODOy[0,1]/np.sin(phi_c)
print(np.sqrt(beta_y*nemit/GAMMA0))

## Beta functions

In [ ]:
FODOx = focus(kf, lf1st) @ \
        drift(L) @ focus(kd, ld) @ drift(L) @ \
        focus(kf, lf1st)
phi_c = (np.arccos(np.trace(FODOx)/2))
beta_x = FODOx[0,1]/np.sin(phi_c)

sol = solve_ivp(rhs, [0, lf1st], [np.sqrt(beta_x), 0, 0], args=[kf], max_step=0.001)

z = list(sol.t)
betax = list(sol.y[0]**2)

sol = solve_ivp(rhs, [0, L], [sol.y[i,-1] for i in range(3)], args=[0], max_step=0.01)
z += list(sol.t + z[-1])[1:]
betax += list(sol.y[0]**2)[1:]

sol = solve_ivp(rhs, [0, ld], [sol.y[i,-1] for i in range(3)], args=[kd], max_step=0.001)
z += list(sol.t + z[-1])[1:]
betax += list(sol.y[0]**2)[1:]

sol = solve_ivp(rhs, [0, L], [sol.y[i,-1] for i in range(3)], args=[0], max_step=0.01)
z += list(sol.t + z[-1])[1:]
betax += list(sol.y[0]**2)[1:]

sol = solve_ivp(rhs, [0, lf1st], [sol.y[i,-1] for i in range(3)], args=[kf], max_step=0.001)
plt.plot(sol.t, np.sqrt(sol.y[0]**2*nemit/GAMMA0), '.-')
plt.title('The final quad integration')
plt.xlabel('Integration distance, m')
plt.ylabel(r'$\sigma_x,\ \mu m$')
plt.show()
z += list(sol.t + z[-1])[1:]
betax += list(sol.y[0]**2)[1:]
plt.plot(z, betax)
plt.xlabel('Integration distance, m')
plt.ylabel(r'$\beta_x,\ m$')
plt.xlim([0, z[-1]])
plt.show()

print(f"The phase advance is {sol.y[2,-1]} rad.")
bx_av = simps(betax, z)/z[-1]
print(f"Average x beta function is {bx_av} m.")

print(f"The maximum x beta function is {np.max(betax)} m; the minimum x beta function is {np.min(betax)} m.")
print(f"Arithmetic average beta function is {0.5*(np.max(betax)+np.min(betax))} m.")

In [ ]:
FODOy = focus(-kf+Ku, lf1st) @ \
        focus(Ku, L) @ focus(-kd+Ku, ld) @ focus(Ku, L) @ \
        focus(-kf+Ku, lf1st)
phi_c = (np.arccos(np.trace(FODOy)/2))
beta_y = FODOy[0,1]/np.sin(phi_c)

sol = solve_ivp(rhs, [0, lf1st], [np.sqrt(beta_y), 0, 0], args=[-kf+Ku], max_step=0.001)

z = list(sol.t)
betay = list(sol.y[0]**2)

sol = solve_ivp(rhs, [0, L], [sol.y[i,-1] for i in range(3)], args=[Ku], max_step=0.01)
z += list(sol.t + z[-1])[1:]
betay += list(sol.y[0]**2)[1:]

sol = solve_ivp(rhs, [0, ld], [sol.y[i,-1] for i in range(3)], args=[-kd+Ku], max_step=0.001)
z += list(sol.t + z[-1])[1:]
betay += list(sol.y[0]**2)[1:]

sol = solve_ivp(rhs, [0, L], [sol.y[i,-1] for i in range(3)], args=[Ku], max_step=0.01)
z += list(sol.t + z[-1])[1:]
betay += list(sol.y[0]**2)[1:]

sol = solve_ivp(rhs, [0, lf1st], [sol.y[i,-1] for i in range(3)], args=[-kf+Ku], max_step=0.001)
plt.plot(sol.t, np.sqrt(sol.y[0]**2*nemit/GAMMA0), '.-')
plt.title('The final quad integration')
plt.xlabel('Integration distance, m')
plt.ylabel(r'$\sigma_y,\ \mu m$')
plt.show()
z += list(sol.t + z[-1])[1:]
betay += list(sol.y[0]**2)[1:]
plt.plot(z, betay)
plt.xlabel('Integration distance, m')
plt.ylabel(r'$\beta_y,\ m$')
plt.xlim([0, z[-1]])
plt.show()

print(f"The phase advance is {sol.y[2,-1]} rad.")
by_av = simps(betay, z)/z[-1]
print(f"Average y beta function is {by_av} m.")

print(f"The maximum y beta function is {np.max(betay)} m; the minimum y beta function is {np.min(betay)} m.")
print(f"Arithmetic average beta function is {0.5*(np.max(betay)+np.min(betay))} m.")

In [ ]:
kd

In [ ]:
kf